In [1]:
from importlib import reload
import sys, os
import re
import copy
from pathlib import Path
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version
import itertools
import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from AMI_SQL import AMI_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
from GenAn import GenAn
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [2]:
def assess_deltas_for_df_i(
    df_i, 
    outg_rec_nb_col = 'OUTG_REC_NB', 
    crt_dt_col      = 'CRT_DT', 
    crt_dt_ts_col   = 'CRT_DT_TS', 
    field_nm_col    = 'FIELD_NM', 
    old_val_tx_col  = 'OLD_VAL_TX', 
    new_val_tx_col  = 'NEW_VAL_TX', 
    verbose         = True
):
    r"""
    Designed to work inside of groupby.apply call!
    """
    #--------------------------------------------------
    assert(df_i[outg_rec_nb_col].nunique()==1)
    assert(df_i[crt_dt_col].nunique()==1)
    #-----
    outg_rec_nb     = df_i[outg_rec_nb_col].unique().tolist()[0]
    warning_tripped = False
    #--------------------------------------------------
    # At most, the field_nm_col column should contain the following values: ['Comments', 'Tot Cust Min Interr', 'Tot Cust Interr']
    #   'Comments' must be contained
    assert(set(df_i[field_nm_col].unique()).difference(set(['Comments', 'Tot Cust Min Interr', 'Tot Cust Interr']))==set())
    
    #--------------------------------------------------
    assert('Comments' in df_i[field_nm_col].unique().tolist())
    #-----
    comment_old_new   = df_i.loc[df_i[field_nm_col]=='Comments'][[old_val_tx_col, new_val_tx_col]]
    assert(comment_old_new.shape[0]==1)
    comment_old   = comment_old_new.iloc[0][old_val_tx_col]
    comment_new   = comment_old_new.iloc[0][new_val_tx_col]
    #-----
    assert('XJXX' in comment_new.upper())
    
    #--------------------------------------------------
    if 'Tot Cust Min Interr' in df_i[field_nm_col].unique().tolist():
        cmi_old_new   = df_i.loc[df_i[field_nm_col]=='Tot Cust Min Interr'][[old_val_tx_col, new_val_tx_col, crt_dt_ts_col]]
        # I suppose the content of the else statement below would handle both cases, but this makes it more explicitly clear what is going on
        if cmi_old_new.shape[0]==1:
            cmi_old = float(cmi_old_new.iloc[0][old_val_tx_col])
            cmi_new = float(cmi_old_new.iloc[0][new_val_tx_col])
        else:
            cmi_old = float(cmi_old_new.sort_values(by=[crt_dt_ts_col], ascending=True).iloc[0][old_val_tx_col])
            cmi_new = float(cmi_old_new.sort_values(by=[crt_dt_ts_col], ascending=True).iloc[-1][new_val_tx_col])

        #-----
        delta_cmi = cmi_old-cmi_new
    else:
        if verbose:
            print(f'Warning: Unable to calculate delta_cmi for OUTG_REC_NB = {outg_rec_nb}')
        warning_tripped = True
        cmi_old   = np.nan
        cmi_new   = np.nan
        delta_cmi = 0
        
    #--------------------------------------------------
    if 'Tot Cust Interr' in df_i[field_nm_col].unique().tolist():
        ci_old_new   = df_i.loc[df_i[field_nm_col]=='Tot Cust Interr'][[old_val_tx_col, new_val_tx_col, crt_dt_ts_col]]
        # I suppose the content of the else statement below would handle both cases, but this makes it more explicitly clear what is going on
        if ci_old_new.shape[0]==1:
            ci_old = float(ci_old_new.iloc[0][old_val_tx_col])
            ci_new = float(ci_old_new.iloc[0][new_val_tx_col])
        else:
            ci_old = float(ci_old_new.sort_values(by=[crt_dt_ts_col], ascending=True).iloc[0][old_val_tx_col])
            ci_new = float(ci_old_new.sort_values(by=[crt_dt_ts_col], ascending=True).iloc[-1][new_val_tx_col])            
        #-----
        delta_ci = ci_old-ci_new
    else:
        if verbose:
            print(f'Warning: Unable to calculate delta_ci for OUTG_REC_NB = {outg_rec_nb}')
        warning_tripped = True
        ci_old   = np.nan
        ci_new   = np.nan
        delta_ci = 0

    #--------------------------------------------------
    
    return_srs = pd.Series({
        'cmi_old'       : cmi_old, 
        'cmi_new'       : cmi_new, 
        'delta_cmi'     : delta_cmi, 
        #-----
        'ci_old'        : ci_old, 
        'ci_new'        : ci_new, 
        'delta_ci'      : delta_ci, 
    })
    #--------------------------------------------------
    if warning_tripped and verbose:
        print('\n')
    #--------------------------------------------------
    return return_srs

In [3]:
def get_dates_01_from_dir_name(
    name
):
    r"""
    Super simple function.
    name must be, e.g., '20230806_20230812'
    """
    #-------------------------
    dates = name.split('_')
    assert(len(dates)==2)
    date_0 = dates[0]
    date_1 = dates[1]
    #-------------------------
    return pd.to_datetime(date_0), pd.to_datetime(date_1)


def amass_ci_cmi_summaries(
    summary_paths, 
    return_included  = False,
    save_dir         = None, 
    date_in_fname    = True
    
):
    r"""
    summary_paths:
        Can be a list of paths or a dict with (name,path) (key,value) pairs
    """
    #--------------------------------------------------
    assert(Utilities.is_object_one_of_types(summary_paths, [list, dict]))
    if isinstance(summary_paths, list):
        summary_paths = {x:x for x in summary_paths}
    #--------------------------------------------------
    included_dict    = dict()
    ci_cmi_summaries = []
    for name_i, summary_path_i in summary_paths.items():
        assert(name_i not in included_dict.keys())
        if os.path.exists(summary_path_i):
            ci_cmi_summary_i = pd.read_pickle(summary_path_i)
            ci_cmi_summaries.append(ci_cmi_summary_i)
            included_dict[name_i] = True
        else:
            included_dict[name_i] = False
    #--------------------------------------------------
    cols = ci_cmi_summaries[0].columns.tolist()
    for ci_cmi_summary_i in ci_cmi_summaries:
        assert(ci_cmi_summary_i.columns.tolist()==cols)
    ci_cmi_summary = pd.concat(ci_cmi_summaries, axis=0)
    #--------------------------------------------------
    if save_dir is not None:
        save_path = os.path.join(save_dir, 'ci_cmi_summary.pkl')
        if date_in_fname:
            date_str  = datetime.datetime.today().strftime('%Y%m%d')
            appndx    = f'_Updated{date_str}'
            #-----
            save_path = Utilities.append_to_path(
                save_path                     = save_path, 
                appendix                      = appndx, 
                ext_to_find                   = '.pkl', 
                append_to_end_if_ext_no_found = False
            )
        #-------------------------
        ci_cmi_summary.to_pickle(save_path)
    #--------------------------------------------------
    if return_included:
        return ci_cmi_summary, included_dict
    else:
        return ci_cmi_summary

def amass_outg_rec_nb_to_files_dicts(
    results_dirs, 
    opcos            = None, 
    dict_fname       = 'outg_rec_nb_to_files_dict.pkl', 
    return_included  = False,
    save_dict_to_pkl = False, 
    date_in_fname    = True
    
):
    r"""
    results_dirs:
        All paths in results_dirs are expected to share the same parent directory!
    
    opcos:
        For now, if not None, must be simple string.
        So, e.g., 'oh' or 'swp'
    """
    #--------------------------------------------------
    # All paths in results_dirs are expected to share the same parent directory!
    parent_dir = Path(results_dirs[0]).parent
    for dir_i in results_dirs:
        assert(Path(dir_i).parent == parent_dir)
    #--------------------------------------------------
    assert(opcos is None or isinstance(opcos, str))
    if opcos is None:
        opcos = 'AllOPCOs'
    #-------------------------
    included_dict             = dict()
    outg_rec_nb_to_files_dict = dict()
    for dir_i in results_dirs:
        assert(dir_i not in included_dict.keys())
        base_dir_dict_i = os.path.join(dir_i, opcos)
        if os.path.exists(os.path.join(base_dir_dict_i, dict_fname)):
            with open(os.path.join(base_dir_dict_i, dict_fname), 'rb') as handle:
                outg_rec_nb_to_files_dict_i = pickle.load(handle)
                #-----
                assert(set(outg_rec_nb_to_files_dict_i.keys()).intersection(set(outg_rec_nb_to_files_dict.keys()))==set())
                outg_rec_nb_to_files_dict = outg_rec_nb_to_files_dict | outg_rec_nb_to_files_dict_i
            included_dict[dir_i] = True
        else:
            included_dict[dir_i] = False
    #--------------------------------------------------
    if save_dict_to_pkl:
        save_path = os.path.join(parent_dir, dict_fname)
        appndx     = f'_{opcos}'
        if date_in_fname:
            date_str  = datetime.datetime.today().strftime('%Y%m%d')
            appndx += f'_Updated{date_str}'
        save_path = Utilities.append_to_path(
            save_path                     = save_path, 
            appendix                      = appndx, 
            ext_to_find                   = '.pkl', 
            append_to_end_if_ext_no_found = False
        )
        #-------------------------
        with open(save_path, 'wb') as handle:
            pickle.dump(outg_rec_nb_to_files_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #--------------------------------------------------
    if return_included:
        return outg_rec_nb_to_files_dict, included_dict
    else:
        return outg_rec_nb_to_files_dict


def get_results_dirs(
    base_dir
):
    r"""
    """
    #-------------------------
    # Basic glob pattern: r'[0-9]*_[0-9]*'
    glob_digit       = r'[0-9]'
    n_digits_in_date = 8
    #-----
    glob_date    = glob_digit*n_digits_in_date
    glob_pattern = glob_date + '_' + glob_date
    #-------------------------
    results_dirs = Utilities.find_all_paths(
        base_dir          = base_dir, 
        glob_pattern      = glob_pattern, 
        regex_pattern     = None, 
        regex_ignore_case = False, 
        recursive         = False
    )
    #-------------------------
    return results_dirs


def get_summary_paths(
    results_dirs, 
    subdirs         = r'AllOPCOs\Results', 
    fname           = r'ci_cmi_summary.pkl', 
    verbose         = True,
):
    r"""
    """
    #-------------------------
    if subdirs is None:
        subdirs = ''
    #-------------------------
    summary_paths = {}
    included_dict = {}
    for result_dir_i in results_dirs:
        assert(result_dir_i not in included_dict.keys())
        assert(os.path.isdir(result_dir_i))
        #-----
        dir_i_name = Path(result_dir_i).name
        #-----
        assert(dir_i_name not in summary_paths.keys())
        #-------------------------
        path_i = os.path.join(result_dir_i, subdirs, fname)
        if os.path.exists(path_i):
            summary_paths[dir_i_name] = os.path.join(result_dir_i, subdirs, fname)
            included_dict[result_dir_i] = True
        else:
            included_dict[result_dir_i] = False
    #-------------------------
    if verbose:
        not_included = {k:v for k,v in included_dict.items() if v==False}
        if len(not_included)>0:
            print(f"NOT INCLUDED:")
            for result_dir_i in not_included.keys():
                print(f"\t{result_dir_i}")
    #-------------------------
    return summary_paths


def impose_minmax_dates_on_summary_paths(
    summary_paths, 
    min_date      = None, 
    max_date      = None    
):
    r"""
    summary_paths:
        MUST be a dict with 
            keys   = to the date range of the data 
            values = ci_cmi_summary.pkl path
        e.g., = {
                '20230806_20230812': 'U:\\CloudData\\dovs_check\\Weekly\\20230806_20230812\\AllOPCOs\\Results\\ci_cmi_summary.pkl',
                '20230813_20230819': 'U:\\CloudData\\dovs_check\\Weekly\\20230813_20230819\\AllOPCOs\\Results\\ci_cmi_summary.pkl',
                ...
            }
    """
    #-------------------------
    if min_date is None and max_date is None:
        return summary_paths
    #-------------------------
    if min_date is None:
        min_date = pd.Timestamp.min
    if max_date is None:
        max_date = pd.Timestamp.max
    #-------------------------
    results_to_keep = {
        k:v for k,v in summary_paths.items() 
        if get_dates_01_from_dir_name(k)[0] >= min_date and get_dates_01_from_dir_name(k)[1] <= max_date
    }
    #-------------------------
    return results_to_keep

# As Documented in DOVS

In [5]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
schema_name = 'DOVSADM'
table_name = 'DOVS_AUDIT_LOG_DIM'

In [6]:
cols_to_select = [
    dict(
        field_desc = "to_char(AUDT_LOG_ID)", 
        alias      = "AUDT_LOG_ID")
    , 
    "USER_ID", 
    "OUTG_REC_NB", 
    "CRT_DT_TS", 
    dict(
        field_desc = "TRUNC(CRT_DT_TS)", 
        alias      = "CRT_DT"
    ), 
    "TABLE_NM", 
    "FIELD_NM", 
    "OLD_VAL_TX", 
    "NEW_VAL_TX", 
    "MOD_TYP_TX", 
    "SOURCE_NM", 
    dict(
        field_desc = "TO_CHAR(EDW_LAST_UPDT_DT,'yyyy-mm-dd hh24:mi:ss.ff')", 
        alias      = "EDW_LAST_UPDT_DT"
    ), 
    dict(
        field_desc = "to_char(MODIFICATION_NB)", 
        alias      = "MODIFICATION_NB"
    ), 
    "OUTAGE_NB"
]
#--------------------------------------------------
sql_select = SQLSelect(field_descs=cols_to_select)
#--------------------------------------------------

In [7]:
sql_1 = """
{} 
FROM DOVSADM.DOVS_AUDIT_LOG_DIM
WHERE NEW_VAL_TX LIKE '%XJXX%'
AND   OLD_VAL_TX NOT LIKE '%XJXX%'
""".format(sql_select.get_statement_string())
df_1 = pd.read_sql_query(sql_1, conn_outages, dtype={'OUTG_REC_NB':np.int32}) 
df_1.shape

C:\Temp\ipykernel_4336\3097420592.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_1 = pd.read_sql_query(sql_1, conn_outages, dtype={'OUTG_REC_NB':np.int32})


(1002, 14)

In [8]:
# Not sure why TO_DATE doesn't work below, but TRUNC does
sql_2 = """
{} 
FROM DOVSADM.DOVS_AUDIT_LOG_DIM
WHERE OUTG_REC_NB IN ({})
AND   TRUNC(CRT_DT_TS) IN ({})
AND   FIELD_NM IN ('Comments', 'Tot Cust Min Interr', 'Tot Cust Interr')
""".format(
    sql_select.get_statement_string(), 
    Utilities_sql.join_list(df_1['OUTG_REC_NB'].unique().tolist(), quotes_needed=True), 
    Utilities_sql.join_list(df_1['CRT_DT_TS'].dt.date.unique().tolist(), quotes_needed=True)
)
df_2 = pd.read_sql_query(sql_2, conn_outages, dtype={'OUTG_REC_NB':np.int32, 'AUDT_LOG_ID':np.int32, 'MODIFICATION_NB':np.int32}) 
df_2.shape

C:\Temp\ipykernel_4336\3480073307.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql_query(sql_2, conn_outages, dtype={'OUTG_REC_NB':np.int32, 'AUDT_LOG_ID':np.int32, 'MODIFICATION_NB':np.int32})


DatabaseError: Execution failed on sql '
SELECT
	to_char(AUDT_LOG_ID) AS AUDT_LOG_ID,
	USER_ID,
	OUTG_REC_NB,
	CRT_DT_TS,
	TRUNC(CRT_DT_TS) AS CRT_DT,
	TABLE_NM,
	FIELD_NM,
	OLD_VAL_TX,
	NEW_VAL_TX,
	MOD_TYP_TX,
	SOURCE_NM,
	TO_CHAR(EDW_LAST_UPDT_DT,'yyyy-mm-dd hh24:mi:ss.ff') AS EDW_LAST_UPDT_DT,
	to_char(MODIFICATION_NB) AS MODIFICATION_NB,
	OUTAGE_NB 
FROM DOVSADM.DOVS_AUDIT_LOG_DIM
WHERE OUTG_REC_NB IN ('13736048','13734808','13737892','13787464','13787770','13784814','13782382','13787496','13787888','13784623','13787013','13785489','13788614','13788151','13787981','13785170','13786448','14094479','14095327','14095749','14095916','14094106','14093030','14093053','14093831','14094673','14094956','14095148','14092885','14092759','14095477','14094913','14094478','14094390','14093389','14093524','14093560','14093787','14093694','14093723','14093880','14094717','14095244','14094063','14105246','14095945','14096327','14096103','14096137','14104414','14105975','14106194','14107225','14096

In [28]:
df_1['OUTG_REC_NB'].nunique()

1002

In [ ]:
#--------------------------------------------------
# The following is for safety, but in most cases it likely has no effect
#-----
# NOTE:  df_2.groupby(['OUTG_REC_NB', 'CRT_DT']).filter(lambda x: x.name in outg_rec_nb_crt_dt_pairs)
#        df_2.groupby(['OUTG_REC_NB', 'CRT_DT'], as_index=False, group_keys=False).apply(lambda x: x if x.name in outg_rec_nb_crt_dt_pairs else None)
#--------------------------------------------------
outg_rec_nb_crt_dt_pairs = df_1[['OUTG_REC_NB', 'CRT_DT']].values.tolist()
# Get unique pairs
outg_rec_nb_crt_dt_pairs = list(set([tuple(x) for x in outg_rec_nb_crt_dt_pairs]))
#-------------------------
df_2 = df_2.groupby(['OUTG_REC_NB', 'CRT_DT']).filter(lambda x: x.name in outg_rec_nb_crt_dt_pairs)
#--------------------------------------------------

In [ ]:
#--------------------------------------------------
df_3 = df_2.groupby(['OUTG_REC_NB', 'CRT_DT']).apply(lambda x: assess_deltas_for_df_i(df_i=x))
#-------------------------
# I expect each OUTG_REC_NB to have only been updated by DOVSAudit results on a single day
# This boils down to each 'OUTG_REC_NB' having a single unique CRT_DT
assert(df_3.index.nunique() == df_3.shape[0])
df_3 = df_3.reset_index(drop=False)

In [ ]:
df_3['delta_cmi'].sum()

In [ ]:
df_3['delta_ci'].sum()

In [ ]:
df_3b = DOVSOutages.get_outg_info_for_df(
    df               = df_3, 
    outg_rec_nb_idfr = 'OUTG_REC_NB', 
    build_sql_function = DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs = dict(cols_of_interest = ['DT_OFF_TS', 'OUTG_REC_NB'])
)

In [ ]:
# df_3b = DOVSOutages.append_outg_info_to_df(
#     df               = df_3b.set_index('OUTG_REC_NB'), 
#     outg_rec_nb_idfr = 'index', 
#     build_sql_function = DOVSOutages_SQL.build_sql_outage, 
#     build_sql_function_kwargs = dict(cols_of_interest = ['DT_OFF_TS', 'OUTG_REC_NB'])
# )

# My Results

In [ ]:
results_dirs = get_results_dirs(base_dir = r'U:\CloudData\dovs_check\Weekly')
# min_date = None
# max_date = None

min_date = df_3b['DT_OFF_TS'].min()
max_date = df_3b['DT_OFF_TS'].max()

In [ ]:
summary_paths = get_summary_paths(
    results_dirs, 
    subdirs         = r'AllOPCOs\Results', 
    fname           = r'ci_cmi_summary.pkl', 
    verbose         = True,
)
#-------------------------
results_to_keep = impose_minmax_dates_on_summary_paths(
    summary_paths, 
    min_date      = min_date, 
    max_date      = max_date    
)
#--------------------------------------------------
ci_cmi_summary, included_dict_summaries = amass_ci_cmi_summaries(
    summary_paths    = results_to_keep, 
    return_included  = True,
    save_dir         = None, 
    date_in_fname    = True
)
#-------------------------
delta_ci_net  = ci_cmi_summary['delta_ci_dovs_ami'].sum()
delta_cmi_net = ci_cmi_summary['delta_cmi_dovs_ami'].sum()
#-----
delta_ci_net_dovs_beg  = ci_cmi_summary['delta_ci_dovs_ami_dovs_beg'].sum()
delta_cmi_net_dovs_beg = ci_cmi_summary['delta_cmi_dovs_ami_dovs_beg'].sum()


#--------------------------------------------------
ci_cmi_summary_all, included_dict_summaries_all = amass_ci_cmi_summaries(
    summary_paths    = summary_paths, 
    return_included  = True,
    save_dir         = None, 
    date_in_fname    = True
)
#-------------------------
delta_ci_net_all  = ci_cmi_summary_all['delta_ci_dovs_ami'].sum()
delta_cmi_net_all = ci_cmi_summary_all['delta_cmi_dovs_ami'].sum()
#-----
delta_ci_net_dovs_beg_all  = ci_cmi_summary_all['delta_ci_dovs_ami_dovs_beg'].sum()
delta_cmi_net_dovs_beg_all = ci_cmi_summary_all['delta_cmi_dovs_ami_dovs_beg'].sum()

In [ ]:
# outg_rec_nb_to_files_dict, included_dict_ntf_dct = amass_outg_rec_nb_to_files_dicts(
#     results_dirs     = results_dirs, 
#     opcos            = None, 
#     dict_fname       = 'outg_rec_nb_to_files_dict.pkl', 
#     return_included  = True,
#     save_dict_to_pkl = False, 
#     date_in_fname    = True
# )

In [ ]:
df_3['OUTG_REC_NB'].astype(str)

In [ ]:
set(df_3['OUTG_REC_NB'].astype(str).unique()).difference(set(list(outg_rec_nb_to_files_dict.keys())))

In [ ]:
df_3[df_3['OUTG_REC_NB']==13806327]

In [ ]:
ci_cmi_summary_overlap = ci_cmi_summary[ci_cmi_summary['outg_rec_nb'].isin(df_3['OUTG_REC_NB'].astype(str).unique())].copy()
ci_cmi_summary_overlap

In [ ]:
ci_cmi_summary_overlap['delta_ci_dovs_ami_dovs_beg'].sum()

In [ ]:
ci_cmi_summary_overlap['delta_cmi_dovs_ami_dovs_beg'].sum()

In [ ]:
print(f"Results supplied (All)")
print(f"delta_ci:  {np.round(delta_ci_net_dovs_beg_all, 2)}")
print(f"delta_cmi: {np.round(delta_cmi_net_dovs_beg_all, 2)}")
print()
print(f"Results supplied ({min_date.date()} - {max_date.date()})")
print(f"delta_ci:  {np.round(delta_ci_net_dovs_beg, 2)}")
print(f"delta_cmi: {np.round(delta_cmi_net_dovs_beg, 2)}")
print()
print(f"Results changed in DOVS and marked XJXX ({min_date.date()} - {max_date.date()})")
print(f"delta_ci:  {np.round(df_3['delta_ci'].sum(), 2)}")
print(f"delta_cmi: {np.round(df_3['delta_cmi'].sum(), 2)}")
print()
print(f"Results supplied (Including only those OUTG_REC_NBs changed in DOVS)")
print(f"delta_ci:  {np.round(ci_cmi_summary_overlap['delta_ci_dovs_ami_dovs_beg'].sum(), 2)}")
print(f"delta_cmi: {np.round(ci_cmi_summary_overlap['delta_cmi_dovs_ami_dovs_beg'].sum(), 2)}")
print()